# Tugas 1 : Crawling Berita

## Adz Dzikry Pradana Putra - 210411100093
## Penambangan Web A

### 1. Import library yang dibutuhkan untuk melakukan web crawl

* Menggunakan BeautifulSoup, konten HTML dari halaman web diurai untuk mencari elemen artikel yang relevan.

* Menggunakan pandas Library digunakan untuk membuat dan mengelola DataFrame.

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

### 2. Membuat Variable kosomg yang berfungsi untuk menyimpan data yang diambil dari berita

In [18]:
# Mendefinisikan list untuk menyimpan data yang diambil
judul = []
isi = []
tanggal = []
url_list = []
kategori_list = []

### 3. Membuat fungsi untuk memfilter konten dari elemen-elemen HTML pada berita yang tidak diinginkan , Contoh kasus seperti iklan, daftar isi, gambar , link sisipan, dll

In [19]:
# Fungsi untuk membersihkan konten dari elemen-elemen yang tidak diinginkan
def clean_content(content_element):
    if content_element:
        # Hapus elemen yang berisi daftar isi
        for daftar_isi in ["collapsible"]:
            unwanted = content_element.find("div", id=daftar_isi)
            if unwanted:
                unwanted.decompose()

        # Hapus elemen yang berisi tag
        for tag_class in ["aevp", "detail__body-tag mgt-16"]:
            unwanted = content_element.find_all("div", class_=tag_class)
            for el in unwanted:
                el.decompose()

        # Hapus elemen yang berisi link sisipan
        link_sisip = content_element.find_all("table", class_="linksisip")
        for table in link_sisip:
            table.decompose()

        # Hapus elemen paragraf dan span dengan class 'para_caption'
        unwanted_paragraphs = content_element.find_all(["p", "span"], class_="para_caption")
        for para in unwanted_paragraphs:
            para.decompose()

        # Kembalikan teks yang tersisa
        return content_element.get_text(separator=' ', strip=True).strip()

    return "Content Not Found"

### 4. Membuat fungsi untuk melakukan crawling data pada situs web Detik.com. Fungsi ini mengambil data berupa judul berita, tanggal publikasi, isi berita dan kategori berita yang terdapat di halaman tersebut.



* mencari semua elemen-elemen yang mengandung isi penting pada artikel seperti elemen yang berisi judul, tanggal, isi dan kategori berita
* Judul, tanggal, dan isi artikel yang valid kemudian disimpan ke dalam list judul, tanggal, isi dan kategori berita.

In [20]:
# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori, max_articles):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= max_articles:
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")

        # Bersihkan konten menggunakan fungsi clean_content
        content = clean_content(content_element)

        judul.append(title)
        tanggal.append(date)
        isi.append(content)
        url_list.append(link)
        kategori_list.append(kategori)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

### 5. Inisialisasi URL dan Kategori yang Akan Di-Crawl

* max_article = variable ini digunakan sebagai batas maksimum berita yang akan diambil

In [21]:
# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://inet.detik.com/games/indeks"]
categories = ["Games"]

max_articles = 10

### 6. Melakukan Iterasi untuk Setiap URL dan Kategori yang akan di Crawl

In [22]:
# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    if len(judul) >= max_articles:
        break

    # Looping untuk beralih halaman
    for page in range(1, 4):
        if len(judul) >= max_articles:
            break

        url = f"{base_url}/{page}"
        get_data(url, category, max_articles)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

Cara Main Game Memori Google, Kenali Juga Manfaatnya untuk Otak
10 Game Offline Android Terbaik 2024, Dijamin Seru Banget
King Indo Gokil! Santai Banget Lawan Jerman di FIFAe World Cup 2024
Concord Game Baru Sony Dinilai Gagal, Pemainnya Dikit Banget
3 Game Gratis PS Plus di PS4 dan PS5, Ada Little Nightmares II
Cheat GTA San Andreas Bahasa Indonesia di PS4, PS5, PC, dan Android
Daftar Game PS4, PS5, Xbox, Nintendo Switch, dan PC RIlis September 2024
Hasil dan Klasemen FFWS SEA 2024 Fall Week 3: Buriram Salip RRQ Kazu
10 Game Offline Seru 2024 untuk HP, Gratis dan Tanpa Perlu Kuota
Bangga! Indonesia Juara FIFAe World Cup 2024 Football Manager


### 7. Menyimpan hasil Crawl ke CSV , lalu menampilkan hasil Crawl menggunakan Dataframe

In [23]:
# Membuat dataframe dari list data
df = pd.DataFrame({"judul": judul, "isi": isi, "tanggal": tanggal, "kategori": kategori_list})

# Menyimpan dataframe ke file csv
df.to_csv("data_berita_detik.csv", index=False)
df=pd.read_csv("data_berita_detik.csv")
df

,judul,isi,tanggal,kategori
0,"Cara Main Game Memori Google, Kenali Juga Manf...",Jakarta - Kalian udah pernah coba main game me...,"Rabu, 04 Sep 2024 13:33 WIB",Games
1,"10 Game Offline Android Terbaik 2024, Dijamin ...",Jakarta - Mencari game offline Android terbaik...,"Rabu, 04 Sep 2024 11:15 WIB",Games
2,King Indo Gokil! Santai Banget Lawan Jerman di...,Jakarta - Ada pemandangan menarik saat Indones...,"Selasa, 03 Sep 2024 20:03 WIB",Games
3,"Concord Game Baru Sony Dinilai Gagal, Pemainny...","Jakarta - Sony sepertinya harus putar otak, su...","Selasa, 03 Sep 2024 14:00 WIB",Games
4,"3 Game Gratis PS Plus di PS4 dan PS5, Ada Litt...",Jakarta - Sony PlayStation akhirnya mengungkap...,"Selasa, 03 Sep 2024 10:05 WIB",Games
5,"Cheat GTA San Andreas Bahasa Indonesia di PS4,...",Jakarta - Ini dia Cheat GTA San Andreas Bahasa...,"Selasa, 03 Sep 2024 08:39 WIB",Games
6,"Daftar Game PS4, PS5, Xbox, Nintendo Switch, d...","Jakarta - Sudah masuk September nih, kira-kira...","Selasa, 03 Sep 2024 07:37 WIB",Games
7,Hasil dan Klasemen FFWS SEA 2024 Fall Week 3: ...,Jakarta - Pekan ketiga babak knockout stage FF...,"Senin, 02 Sep 2024 16:10 WIB",Games
8,"10 Game Offline Seru 2024 untuk HP, Gratis dan...",Jakarta - Game offline selalu jadi pilihan fav...,"Senin, 02 Sep 2024 13:19 WIB",Games
9,Bangga! Indonesia Juara FIFAe World Cup 2024 F...,Jakarta - Indonesia menjadi juara dunia FIFAe ...,"Senin, 02 Sep 2024 09:45 WIB",Games
